# COLX 535 Lab Assignment 3: CFG parsing algorithms (Cheat sheet)

## Assignment Objectives

In this assignment you will:
- Implement the CYK parsing algorithm
- Use probabilities to pick the best parse
- Practice chart parsing 

## Getting started

Run the following code:

In [1]:
from nltk.parse.earleychart import EarleyChartParser
from nltk.grammar import Nonterminal
from nltk import CFG,PCFG
from nltk.corpus import treebank
from nltk.tree import Tree
from collections import defaultdict
import numpy as np

## Tidy submission

rubric={mechanics:1}

### Exercise 1: CYK parsing

In the next two exercises you will building a parser.

#### 1.1
rubric={accuracy:2}

Your CYK parser will be a class with an initialization function taking a grammar as argument. We will first write the initialization function for the class. There are three tasks you need to complete:

1. Check whether the grammar you've been given is in Chomsky Normal Form. If it's not, you should convert it to CNF using NLTK library functions (see Monday's practical lecture). Store the grammar as `self.grammar`

2. Build a Python dictionary `self.lexical_rules` which allows you to look up LHS of lexical rules based on their RHS (which will be a word form). For example, if your grammar contains the rules:
```
NP -> DT N
VP -> V NP
N -> 'look'
V -> 'look'
N -> 'cat'
DT -> 'a'
```
You would have a dictionary like this:
```
{
 'look': [V, N],
 'house': [N],
 'a' : [DT]
}
```
Note that the non-terminals e.g. `V` and `DT` are not strings. They are `nltk.grammar.Nonterminal` objects.

3. Build a Python dictionary `self.grammatical_rules` which allows you to look up LHS of grammatical rules based on their RHS (which will be a word form). For our example grammar, the dictionary would look like:
```
{
(DT,N):[NP],
(V,NP):[VP]
}
```

**Why are we doing this?** When working with realistic grammars, you can't afford to be scanning through all the rules to find one that applies, you need the fast look-up of a hash map like Python dictionaries. 

In [2]:
from collections import defaultdict

class CYKParser(object):
    
    def __init__(self, grammar):
        # your code here

        # your code here
            
    def parse(self,sentence):
        """ No need to do anything about parse() yet. """
        pass

Assertions to test your initialization function.

In [3]:
dummy_grammar = CFG.fromstring('''
S -> S2
S2 -> NP VP PP
NP -> "dog"
VP -> "dog"
''')

dummy_parser = CYKParser(dummy_grammar)

assert dummy_parser.grammar.is_chomsky_normal_form()
assert (Nonterminal('VP'),Nonterminal('PP')) in dummy_parser.grammatical_rules
assert 'dog' in dummy_parser.lexical_rules
assert set(dummy_parser.lexical_rules["dog"]) == set([Nonterminal("NP"), Nonterminal("VP")])
assert dummy_parser.grammatical_rules[(Nonterminal('VP'), Nonterminal('PP'))] == [Nonterminal('S2@$@NP')]
print('Success!')

dummy_grammar (CFG):
Grammar with 4 productions (start state = S)
    S -> S2
    S2 -> NP VP PP
    NP -> 'dog'
    VP -> 'dog'
------------------------

dummy_grammar (CNF):
Grammar with 5 productions (start state = S)
    NP -> 'dog'
    S2 -> NP S2@$@NP
    S2@$@NP -> VP PP
    S -> NP S2@$@NP
    VP -> 'dog'
Success!


#### 1.2
rubric={accuracy:2}

Write a function `create_table` which builds the table data structure. It should be a Python dictionary where the keys are ranges `(i,j)`, where 
```
0 <= i < j <= sent_len
```
For each key `(i,j)`, you should set the value of `table[(i,j)]` to `[]`.

In [6]:
def create_table(sent_len):
    '''Create a table appropriate for CYK parsing. Returns a dictionary of empty lists'''
    table = {}

    #your code here
    
    #your code here
    return table

Assertions to test your `create_table` function.

In [8]:
assert create_table(3) == {(0,1):[], (0,2):[], (0,3):[], (1,2):[], (1,3):[], (2,3):[]}
print('Success!')

Success!


#### 1.3
rubric={accuracy:4,efficiency:2}

In this step, you're going to write the main parsing logic for `CYKParser`. 

Start by copying your initialization code from assignment 1.1 and pasting it into the `__init__` function of `CYKParser`.

Your first actual task is to implement the **function `CYKParser.check_sentence()`**. It takes a sentence as argument and **returns `True` if we can parse the sentence**, meaning that all the words in the sentence are found in the `lexical_rules` dictionary. Otherwise, it returns `False`.

We will now work on **`CYKParser.parse()`**. You should implement code **to fill in the cells `(i,j)` of the parse table `table`**. The existing code iterates through the columns of `table` from left to right and iterates through every column from the bottom to the top. It then checks every split of the **range `(i,j)` into two children `(i,k)` and `(k,j)`**. It is your task to find all combinations of non-terminals `Y` and `Z` from `table[(i,k)]` and `table[(k,j)]` which can be combined into a new non-terminal `X` using a grammatical rule
```
X -> Y Z
```
You should then add the non-terminal `X` to cell `(i,j)` in the parse table. Note that you might add several non-terminals to the parse table in this step.

In [9]:
from collections import defaultdict

class CYKParser(object):
    def __init__(self, grammar):
        # Copy-paste your __init__ function code from 1.1.

        # your code here
        
                
    def check_sentence(self, sentence):
        # Check that all of the words in our sentence are in the grammar. 
        # If they aren't return False.
        
        # your code here
        
    
    def parse(self, sentence):
        if not self.check_sentence(sentence):
            return False
            
        # Create parse table
        table = create_table(len(sentence))
        
        # Iterate through the parse table from left to right
        for j in range(1,len(sentence)+1):
            # Add the word form at position j-1 to the parse table
            table[(j-1, j)] += self.lexical_rules[sentence[j-1]]
            
            # Iterate through colum j of the parse table 
            for i in range(j-2, -1, -1):
                
                # Iterate through all fence-posts in the range (i,j)
                for k in range(i+1, j):
                    # your code here
                    
                    # your code here
        
        # Check that we actually managed to generate a sentence node in 
        # the upper right corner of the parse table. This means that the
        # grammar accepts our sentence
        if self.grammar.start() in table[(0,len(sentence))]:
            return True
        else:
            return False


Assertions to check that your parser works as it should.

In [10]:
grammar = CFG.fromstring('''
S -> NP VP
NP -> DT NN
VP -> VB
DT -> 'a'
NN -> 'person'
VB -> 'waits'
''')

sent1 = ['a', 'person', 'waits']
sent2 = ['a', 'person', 'wait']
sent3 = ['waits', 'person', 'a']

parser = CYKParser(grammar)
assert parser.parse(sent1)
assert not parser.parse(sent2)
assert not parser.parse(sent3)
print('Success!')

Grammar with 6 productions (start state = S)
    VB -> 'waits'
    NN -> 'person'
    DT -> 'a'
    NP -> DT NN
    VP -> 'waits'
    S -> NP VP
( 0 , 2 ) <- NP
( 0 , 3 ) <- S
Success!


![1.3](1.3.png)

#### 1.4
rubric={accuracy:1}

Start by copying your initialization code from assignment 1.1 and pasting it into the `__init__` function of `CYKParser`. Then copy your code from 1.3 and paste it into the `parse()` function.

In this assignment, we're working with **a second table object `parse_pointers`**. For each non-terminal that you add to `table` for range `(i,j)` and fence-post `k`, you should **add a tuple `(fence-post, left_child, right_child)` to parse-pointers**. This indicates the phrase boundary and which rule was used to derive the non-terminal in cell `(i,j)`. The `parse()` function will then call `generate_trees` which will generate the set of parse trees for you. 

In [11]:
def _generate_trees(parse_pointers,table,i,j,non_term,sentence):
    if i+1 == j:
        return [Tree(pos,[sentence[i]]) for pos in table[(i,j)]]
    else:
        trees = []
        # Iterate over sets here to avoid duplicate trees in the list.
        for mother, (k, left_label, right_label) in set(zip(table[(i,j)], parse_pointers[(i,j)])):
            if mother != non_term:
                continue
            left_children = _generate_trees(parse_pointers, table, i, k, left_label, sentence)
            right_children = _generate_trees(parse_pointers, table, k, j, right_label, sentence)
            for child1 in left_children:
                for child2 in right_children:
                    if child1.label() == left_label and child2.label() == right_label:
                        trees.append(Tree(mother, [child1, child2]))
        return trees

def generate_trees(parse_pointers,table,sentence):
    """ This is a helper function which generates parse trees based on a parse_pointer table
        and a parse table. """
    return _generate_trees(parse_pointers, table, 0, len(sentence), Nonterminal("S"), sentence)
    
class CYKParser(object):
    def __init__(self, grammar):
        # Copy-paste your __init__ function code from 1.1.
        # your code here
                
    def check_sentence(self, sentence):
        # Copy-paste your code from 1.3
        # your code here
       
    def parse(self, sentence):
        if not self.check_sentence(sentence):
            return []
            
        # Create parse table and pointer table
        table = create_table(len(sentence))
        parse_pointers = create_table(len(sentence))
        
        # Iterate through the parse table from left to right
        for j in range(1,len(sentence)+1):
            # Add the word form at position j-1 to the parse table
            table[(j-1, j)] += self.lexical_rules[sentence[j-1]]
            
            # Iterate through colum j of the parse table 
            for i in range(j-2, -1, -1):
                
                # Iterate through all fence-posts in the range (i,j)
                for k in range(i+1, j):
                    # Start by copy-pasting your code from 1.3, then add code 
                    # for updating the parse_pointers
                    # your code here


        if self.grammar.start() in table[(0,len(sentence))]:
            return generate_trees(parse_pointers,table,sentence)
        else:
            return []


Assertions to check that your `pointer_table` is working as it should.

In [12]:
grammar = CFG.fromstring('''
S -> NP VP
NP -> DT NN
VP -> VBZ NP
DT -> 'a'
NN -> 'person' | 'dog'
VBZ -> 'sees'
''')

sent1 = ['a', 'person', 'sees', 'a', 'dog']
sent2 = ['a', 'person', 'sees']
sent3 = ['dog', 'person', 'sees', 'a', 'person']

parser = CYKParser(grammar)
parses = parser.parse(sent1)
assert len(parses) == 1
str(Tree.fromstring('(S (NP (DT a) (NN person)) (VP (VBZ sees) (NP (DT a) (NN dog))))'))
assert(str(parses[0]) == str(Tree.fromstring('(S (NP (DT a) (NN person)) (VP (VBZ sees) (NP (DT a) (NN dog))))')))
assert parser.parse(sent2) == []
assert parser.parse(sent3) == []

pp_grammar = CFG.fromstring('''
S -> NP VP
NP -> NP PP
NP -> DT NN
VP -> VP PP
VP -> VBZ NP
PP -> IN NP
DT -> 'a'
NN -> 'person' | 'dog'
VBZ -> 'sees'
IN -> 'with'
''')
sent1 = ['a', 'person', 'sees', 'a', 'dog', 'with', 'a', 'person']
parser = CYKParser(pp_grammar)
parses = parser.parse(sent1)
assert len(parses) == 2
print('Success!')

( 0 , 2 ) <- NP | 1 DT NN
( 3 , 5 ) <- NP | 4 DT NN
( 2 , 5 ) <- VP | 3 VBZ NP
( 0 , 5 ) <- S | 2 NP VP


![1.4](1.4.png)

### Exercise 2: PCFG parsing

Now, rather than just outputing either one random parse or all possibile parses, let's see how we could pick the best parse. Let's convert the CFG parser into a PCFG parser. 

#### 2.1
rubric= {accuracy:2}


For the ambiguous sentence `S` provided below (**HINT**: *flies* can be both a noun and a verb, *like* can be both a verb and a preposition), create a corresponding PCFG grammar called `pcfg_grammar` which reflects at least one possible ambiguity. It doesn't matter what the probabilities are as long as all the probabilities for a given LHS symbol add up to one. Make sure it is already in Chomsky normal form, since the conversion we've been using doesn't work for PCFGs.

In [13]:
S = ['time', 'flies', 'like', 'an', 'arrow']

# your code here
from nltk import PCFG

pcfg_grammar = PCFG.fromstring('''S -> NP VP [0.1]
S -> NP VBN [0.2]
S -> NNS VP [0.2]
S -> NNS VBN [0.2]
S -> VP PP [0.2]
''')

In [14]:
assert pcfg_grammar.is_chomsky_normal_form()
parser = CYKParser(pcfg_grammar)
assert len(parser.parse(S)) > 1
print('Success!')

Success!


#### 2.2 Optional
rubric= {accuracy:4,efficiency:1,quality:1}

Write a `best_parse` function which takes to arguments: `trees` which is a list of NLTK syntax trees and `grammar` which is a `PCFG`. The function iterates over trees, and picks the one which is assigned the highest probability by `grammar`. 

To get a good efficiency score, you need to do this in a way that will scale well to thousands of parses from a grammar with thousands of productions. One way to do this is to store your grammar productions in a python dictionary like we did in assignment 1. 

**HINT:** Remember that the probability of a tree is the product of the probabilities of all productions, e.g. `S -> NP VP [0.7]` and `NNS -> 'dogs' [0.2]`, which are used when generating the tree. It can be a good idea to use recursion for computing the probability of a parse tree.

In [15]:
# your code here


You should write your own test cases for `best_parse` (at least two) consisting of lists of two or more parse trees which should be outputs of the PCFG you wrote in assignment 2.1. Make sure that `best_parse` returns the tree which gets the highest probability from your grammar.

Your test cases should involve at trees of at least height 3.

In [16]:
# your code here

print("Success!")

Success!


### Exercise 3: Chart parsing
#### 3.1 
rubric={accuracy:1}

Use NLTK to carry out an Earley chart parse of the sentence below, in `trace=1` mode.

In [17]:
sentence = "the cat slept on the porch".split()

grammar = CFG.fromstring('''
S -> NP VP
NP -> DT NN
VP -> VBD
VP -> VP PP
PP -> IN NP
DT -> 'the'
NN -> 'cat'
NN -> 'porch'
VBD -> 'slept'
IN -> 'on'
''')

#your code here


|. the  . cat  .slept .  on  . the  .porch .|
|[------]      .      .      .      .      .| [0:1] 'the'
|.      [------]      .      .      .      .| [1:2] 'cat'
|.      .      [------]      .      .      .| [2:3] 'slept'
|.      .      .      [------]      .      .| [3:4] 'on'
|.      .      .      .      [------]      .| [4:5] 'the'
|.      .      .      .      .      [------]| [5:6] 'porch'
|>      .      .      .      .      .      .| [0:0] S  -> * NP VP
|>      .      .      .      .      .      .| [0:0] NP -> * DT NN
|>      .      .      .      .      .      .| [0:0] DT -> * 'the'
|[------]      .      .      .      .      .| [0:1] DT -> 'the' *
|[------>      .      .      .      .      .| [0:1] NP -> DT * NN
|.      >      .      .      .      .      .| [1:1] NN -> * 'cat'
|.      [------]      .      .      .      .| [1:2] NN -> 'cat' *
|[-------------]      .      .      .      .| [0:2] NP -> DT NN *
|[------------->      .      .      .      .| [0:2] S  -> NP * VP
|.      .

#### 3.2
rubric={raw:3}

For each line in the trace, remove the visual span part of the line and indicate which part of the algorithm produced it and which other line or lines in the trace triggered this line. For instance, a completer line should point to both the version of rule before the dot moved as well as the completed/scanned rule, whereas the predictor should point to the completer line that caused the prediction to be made. You should number the lines for this purposes.  For example,

```
4. PP -> * P NP  Predictor 3
5. P -> 'in' *  Scanner
6. PP -> P * NP Completer 4 5
```

You can remove the lines corresponding to the words in the sentence and unscanned lexical rules such as `P -> * 'in'`.

1.  |>      .      .      .      .      .      .| [0:0] S  -> * NP VP     START
2.  |>      .      .      .      .      .      .| [0:0] NP -> * DT NN     Predictor 1
3.  |[------]      .      .      .      .      .| [0:1] DT -> 'the' *     Scanner
4.  |[------>      .      .      .      .      .| [0:1] NP -> DT * NN     Completer 2 3 
...

## Exercise 4: Teamwork report

#### 4.1

rubric={raw:1, reasoning:1}

Briefly discuss how each person contributed to the project. Though it is not necessary that every group member has a equal contribution in terms of code, every group member should have a significant contribution.

YOUR ANSWER HERE